<div style="text-align: center; font-size: 16px;">
    <strong>Course:</strong> Machine Learning Operations |
    <strong>Lecturer:</strong> Prof. Dr. Klotz |
    <strong>Date:</strong> 17.05.2025 |
    <strong>Name:</strong> Sofie Pischl
</div>

# <center>Data Collection </center>

Konzept & Inhalt:

Daten von den größten Social media Apps sollen abgegriffen werden. besonderer Fokus auf Texten.

1. Setup & Imports
2. Reddit: Hot Posts aus Subreddits
3. Instagram: Top Posts per Scraping/API (light)
4. Twitter: Aktuelle Tweets via snscrape oder Tweepy
5. TikTok: Trending Videos
6. YouTube: Trending Videos (API/Scraping)
7. Fazit & Learnings

----
# 1. Setup & Imports

Import allgemeiner Bibliotheken:

In [27]:
#Standardbibliotheken
import asyncio            # Für asynchrone Programmierung (z. B. async/await)
import csv                # Zum Lesen/Schreiben von CSV-Dateien auf niedriger Ebene
import logging            # Für Logging von Status- und Fehlermeldungen
import os                 # Für Zugriff auf Umgebungsvariablen, Dateipfade etc.
import time               # Für Zeitfunktionen wie sleep()
from datetime import datetime  # Für aktuelle Zeit und Datumsmanipulation
from pathlib import Path  # Für objektorientierten Umgang mit Dateipfaden

#Drittanbieter-Bibliotheken
import pandas as pd       # Für Datenanalyse und CSV-Verarbeitung
from dotenv import load_dotenv  # Zum Laden von Umgebungsvariablen aus .env-Dateien

# Load from .env file
load_dotenv()

True

# 2. Reddit

In [ ]:
import praw  # Reddit API Wrapper für den Zugriff auf Subreddits, Posts und Kommentare

True

### Funktionen:
- Authentifizierung über OAuth2 via `praw`
- Abruf der `hot`-Beiträge aus ausgewählten Subreddits
- Speicherung als `.csv` unter `/data/raw/reddit_data.csv`
- Fehler-Handling und Logging integriert

**Authentifizierung**

Zur Authentifizierung an der Reddit-API wird ein Reddit-Objekt der Bibliothek praw initialisiert. Die benötigten Zugangsdaten – client_id, client_secret und user_agent – werden aus einer .env-Datei geladen, um die Trennung von Code und Konfiguration zu gewährleisten und Sicherheitsrisiken zu minimieren.

Diese Parameter dienen der eindeutigen Identifikation der Anwendung gegenüber der API und sind notwendig, um Zugriff auf Reddit-Inhalte zu erhalten. Der user_agent ermöglicht zudem die Rückverfolgbarkeit von API-Anfragen seitens Reddit. Ohne diese Authentifizierung ist ein reguliertes, automatisiertes Scraping nicht zulässig.

In [2]:
reddit = praw.Reddit(
    client_id=(os.getenv("REDDIT_ID")),
    client_secret=(os.getenv("REDDIT_SECRET")),
    user_agent=(os.getenv("USER_AGENT"))
)

**Logging-Konfiguration**

Bevor das Reddit-Scraping startet, wird ein Logging-System eingerichtet. Dazu wird zunächst ein Pfad zur Log-Datei definiert – in diesem Fall `logs/reddit.log`. Falls das Verzeichnis `logs/` noch nicht existiert, wird es automatisch erstellt. Anschließend wird das Python-Logging so konfiguriert, dass alle Log-Meldungen in diese Datei geschrieben werden.

Die Konfiguration legt fest, dass nur Meldungen ab dem Schweregrad `INFO` gespeichert werden. Außerdem wird das Format der Einträge so definiert, dass jeder Log-Eintrag einen Zeitstempel, den Log-Level (wie `INFO` oder `ERROR`) sowie die eigentliche Nachricht enthält. So entsteht eine nachvollziehbare Chronik über den Ablauf und mögliche Fehler des Scripts.

Ein typischer Eintrag könnte zum Beispiel so aussehen:

```
2025-04-19 14:33:07,512 - INFO - Starte Reddit-Scraping...
```

In [3]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Logging einrichten
log_path = Path("../logs/reddit.log")
log_path.parent.mkdir(parents=True, exist_ok=True)
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
print(f" Logging aktiv unter: {log_path.resolve()}")

 Logging aktiv unter: C:\Users\SofiePischl\Documents\01_HdM\10_ML_OPS\TrendAnalyse Social Media\logs\reddit.log


**Reddit-Datensammlung mittels Python und PRAW**

Die Funktion `scrape_reddit()` dient der systematischen Erhebung textbasierter Inhalte aus der Social-Media-Plattform **Reddit**. Ziel ist es, strukturierte Daten zur Analyse von Trendthemen zu generieren. Zur Umsetzung wird die Bibliothek `praw` (Python Reddit API Wrapper) verwendet, die eine komfortable Schnittstelle zur Reddit-API bereitstellt.

Nach der Initialisierung der Protokollierung via `logging` erfolgt die Authentifizierung an der Reddit-API. Hierfür wird ein `Reddit`-Objekt instanziiert, wobei sensible Zugangsdaten wie `client_id`, `client_secret` und `user_agent` aus einer `.env`-Datei geladen werden. Dieses Vorgehen ermöglicht eine sichere Trennung von Konfiguration und Codebasis und schützt vor dem unbeabsichtigten Leaken von API-Schlüsseln.

Im Anschluss wird eine Liste von Subreddits definiert, die sowohl populäre als auch als „trending“ markierte Communities umfasst. Aus diesen Subreddits werden jeweils bis zu 100 Beiträge aus dem Hot-Feed abgerufen. Dieses Verfahren stellt sicher, dass aktuelle, stark diskutierte Inhalte gesammelt werden, die ein hohes Relevanzpotenzial für Trendanalysen aufweisen.

Die Datenextraktion erfolgt über eine doppelte Schleife: Für jedes Subreddit werden Hot-Beiträge iteriert, wobei ausschließlich „self-posts“ berücksichtigt werden. Diese beinhalten keine externen Links und ermöglichen dadurch eine fokussierte Analyse des vom Nutzer selbst verfassten Textinhalts. Pro Beitrag werden zentrale Metriken wie Titel, Text, Anzahl der Kommentare, Upvotes, Erstellungszeitpunkt sowie die URL gespeichert. Zur besseren zeitlichen Einordnung wird außerdem ein einheitlicher Zeitstempel für alle Einträge vergeben.

Die gesammelten Beiträge werden in einem `pandas.DataFrame` strukturiert und anschließend unter `data/raw/reddit_data.csv` abgespeichert. Dabei wird sichergestellt, dass benötigte Verzeichnisse automatisch erstellt werden. Falls bereits Daten vorhanden sind, werden die neuen Einträge angehängt und anschließend Duplikate basierend auf Titel, Textinhalt und Subreddit entfernt. Die finale Version wird ohne Index in die CSV-Datei geschrieben.

Abschließend wird die Anzahl der gespeicherten Beiträge im Logfile vermerkt. Etwaige Fehler werden während der Ausführung abgefangen und entsprechend protokolliert. Die Funktion kann sowohl als Modul importiert als auch direkt per Skriptausführung genutzt werden.


In [4]:
def scrape_reddit():
    try:
        logging.info("Starte Reddit-Scraping...")

        subreddits = ["all", "popular", "trendingreddits", "trendingsubreddits"]
        post_limit = 100
        all_posts = []
        scrape_time = datetime.now()

        for sub in subreddits:
            subreddit = reddit.subreddit(sub)
            for post in subreddit.hot(limit=post_limit):
                if post.is_self:
                    all_posts.append({
                        "subreddit": sub,
                        "title": post.title,
                        "text": post.selftext,
                        "score": post.score,
                        "comments": post.num_comments,
                        "created": datetime.fromtimestamp(post.created),
                        "url": post.url,
                        "scraped_at": scrape_time
                    })

        df = pd.DataFrame(all_posts)
        csv_path = Path("../app/data/raw/reddit_data.csv")
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        if csv_path.exists():
            df_existing = pd.read_csv(csv_path)
            df = pd.concat([df_existing, df], ignore_index=True)

        df.drop_duplicates(subset=["title", "text", "subreddit"], inplace=True)
        df.to_csv(csv_path, index=False)
        print(f" Gespeichert unter: {csv_path.resolve()}")

        logging.info(f"{len(df)} Einträge gespeichert unter {csv_path}")

    except Exception as e:
        logging.error(f"Fehler beim Reddit-Scraping: {e}")

# Ausführung bei direktem Aufruf
if __name__ == "__main__":
    scrape_reddit()

 Gespeichert unter: C:\Users\SofiePischl\Documents\01_HdM\10_ML_OPS\TrendAnalyse Social Media\app\data\raw\reddit_data.csv


In [7]:
# Zeuge neuste Einträge
df = pd.read_csv("../app/data/raw/reddit_data.csv")
df.head()

,subreddit,title,text,score,comments,created,url,scraped_at
0,all,AITAH for telling my best friend her marriage ...,This weekend was a disaster...\n\nI 27F have b...,17380,1914,2025-05-07 02:03:00,https://www.reddit.com/r/AITAH/comments/1kgjqg...,2025-05-07 11:03:58.051533
1,all,[Post Game Thread] The Indiana Pacers head bac...,\n||\n|:-:|\n|[](/IND) **120 - 119** [](/CLE)...,6946,2069,2025-05-07 03:48:31,https://www.reddit.com/r/nba/comments/1kgltqu/...,2025-05-07 11:03:58.051533
2,popular,"Americans, how do you feel about the firing of...",NaN,21007,5167,2025-05-06 19:46:16,https://www.reddit.com/r/AskReddit/comments/1k...,2025-05-07 11:03:58.051533
3,popular,Claim your Trailer 2 OG flair here!,Out of the blue we got the second trailer...FI...,12751,46078,2025-05-06 16:43:19,https://www.reddit.com/r/GTA6/comments/1kg68js...,2025-05-07 11:03:58.051533
4,popular,AITA for refusing to let my daughter’s fiancé ...,"So I (M49) might be in the wrong here, but I h...",8064,4265,2025-05-06 18:29:17,https://www.reddit.com/r/AmItheAsshole/comment...,2025-05-07 11:03:58.051533


---

# 3. Instagram

Im Rahmen der Datenerhebung für Instagram wurden zunächst etablierte Bibliotheken wie *Instaloader* und *Playwright* getestet. Beide Ansätze erwiesen sich jedoch als unzureichend: *Instaloader* ermöglicht nur den Zugriff auf öffentlich verfügbare Inhalte registrierter Nutzer und bietet keinen Zugriff auf die dynamisch generierte Explore-Seite. *Playwright*, obwohl für modernes Web-Scraping geeignet, stieß auf technische Einschränkungen hinsichtlich der Authentifizierung und der zuverlässigen Navigation innerhalb der dynamischen Inhalte Instagrams.

Aus diesem Grund wurde ein alternativer Ansatz mittels *Selenium* implementiert. Dieser verfolgt die Strategie, das Nutzerverhalten im Browser möglichst realitätsnah zu simulieren. Der automatisierte Prozess umfasst dabei das Aufrufen der Instagram-Loginseite, die Anmeldung über Umgebungsvariablen gespeicherte Zugangsdaten, die Navigation zur Explore-Seite sowie das Extrahieren von Beitragsinformationen (z. B. Bildquelle, Beschreibung, URL) über XPath-Selektoren.

Grundsätzlich zeigte sich der Selenium-basierte Ansatz als funktional, jedoch mit erheblichen Stabilitätsproblemen. Da die DOM-Struktur der Explore-Seite nicht statisch ist, können sich XPath-Referenzen zwischenzeitlich ändern oder variieren, was zu fehleranfälligen Selektionsoperationen führt. Zudem ist die Explore-Seite stark dynamisch und in hohem Maße vom Nutzerverhalten und den Algorithmus-basierten Inhalten abhängig, was eine konsistente und reproduzierbare Datenextraktion erschwert. Somit stellt die Nutzung von Selenium eine technisch mögliche, jedoch fragile und wartungsintensive Lösung für das Scraping dynamischer Instagram-Inhalte dar.

Deshalb wird der Ansatz im folgenden dargestellt, wurde aber im weiteren Verlauf des Projekts verworden.

In [ ]:
from selenium import webdriver  # Startet und steuert Browser-Instanzen (z. B. Chrome)
from selenium.webdriver.chrome.service import Service  # Verwaltet den ChromeDriver-Dienst
from selenium.webdriver.common.by import By  # Selektoren zum Finden von Elementen (z. B. By.ID, By.XPATH)
from selenium.webdriver.chrome.options import Options  # Konfiguration für den Chrome-Browser (z. B. headless-Modus)
from selenium.webdriver.support.ui import WebDriverWait  # Ermöglicht explizite Wartezeiten für Elemente
from selenium.webdriver.support import expected_conditions as EC  # Vordefinierte Bedingungen zum Warten (z. B. Sichtbarkeit von Elementen)
from webdriver_manager.chrome import ChromeDriverManager  # Automatische Verwaltung und Installation des ChromeDrivers

True

In [ ]:
#Credentials aus der .env Datei laden
INSTA_USERNAME= os.getenv("INSTA_USERNAME")
INSTA_PASSWORD= os.getenv("INSTA_PASSWORD")

In [22]:
# Logging initialisieren
log_path = Path("../logs/instagram.log")
log_path.parent.mkdir(parents=True, exist_ok=True)
# Logger holen (Root-Logger)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Bestehende Handler entfernen (z. B. alte Datei-Handler)
logger.handlers.clear()

# Neuen FileHandler hinzufügen
file_handler = logging.FileHandler(log_path)

logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

print(f"Logging aktiv unter: {log_path.resolve()}")

Logging aktiv unter: C:\Users\SofiePischl\Documents\01_HdM\10_ML_OPS\TrendAnalyse Social Media\logs\instagram.log


Die folgende Funktion scrape_instagram_explore() automatisiert den Zugriff auf die Instagram-Explore-Seite, um dort eingebettete Beiträge zu extrahieren und zu speichern. Zunächst wird ein Headless-Browser mittels Selenium konfiguriert und gestartet, um Instagram ohne sichtbares Browserfenster zu laden. Anschließend erfolgt ein Login mit Benutzerdaten, die aus Umgebungsvariablen gelesen werden. Nach erfolgreicher Anmeldung wird zur Explore-Seite navigiert, wo durch wiederholtes Scrollen weitere Inhalte dynamisch nachgeladen werden. Die so sichtbaren Beiträge werden dann anhand von XPath-Selektoren identifiziert, und von jedem Beitrag werden die URL, das zugehörige Bild sowie die Bildbeschreibung ausgelesen. Diese Informationen werden zusammen mit einem Zeitstempel gesammelt und in eine CSV-Datei gespeichert. Bestehende Daten in der Datei werden dabei berücksichtigt und Duplikate entfernt. Die Funktion schließt mit dem Speichern der kombinierten Daten und dem Beenden des Browsers. Fehler während des Prozesses werden per Logging dokumentiert.

In [ ]:

def scrape_instagram_explore():
    try:
        logging.info("Starte Instagram-Explore-Scraping...")

        # Headless-Browser konfigurieren
        options = Options()
        options.add_argument("--headless=new")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get("https://www.instagram.com/accounts/login/")

        # Login
        username = os.getenv("INSTA_USERNAME")
        password = os.getenv("INSTA_PASSWORD")

        time.sleep(3)
        driver.find_element(By.NAME, "username").send_keys(username)
        driver.find_element(By.NAME, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()

        time.sleep(7)  # Warte auf Login

        # Gehe zur Explore-Seite
        driver.get("https://www.instagram.com/explore/")
        time.sleep(5)

        # Seite scrollen, um mehr Posts zu laden
        for _ in range(3):  # 3x scrollen → kannst du erhöhen
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

        # Beiträge extrahieren
        posts = driver.find_elements(By.XPATH, '//a[contains(@href, "/p/")]')
        post_data = []

        for post in posts:
            try:
                url = post.get_attribute("href")
                img = post.find_element(By.TAG_NAME, "img")
                img_url = img.get_attribute("src")
                description = img.get_attribute("alt")

                post_data.append({
                    "timestamp": datetime.now().isoformat(),
                    "url": url,
                    "image": img_url,
                    "description": description
                })
            except Exception as e:
                logging.warning(f"Fehler beim Extrahieren eines Posts: {e}")
                continue

        # In CSV speichern
        csv_path = Path("../app/data/raw/instagram_data.csv")
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        df = pd.DataFrame(post_data)

        if csv_path.exists():
            df_existing = pd.read_csv(csv_path)
            df = pd.concat([df_existing, df], ignore_index=True)

        df.drop_duplicates(subset=["url", "image", "description"], inplace=True)
        df.to_csv(csv_path, index=False)

        logging.info(f"{len(df)} Einträge gespeichert unter {csv_path}")
        driver.quit()

    except Exception as e:
        logging.error(f"Fehler beim Instagram-Explore-Scraping: {e}")

In [24]:
# Jetzt ausführen
scrape_instagram_explore()

Starte Instagram-Explore-Scraping...


Zur gezielten Extraktion von Informationen aus einzelnen Instagram-Posts wurde ein browsergestützter Ansatz mittels Selenium implementiert. Die Funktion scrape_instagram_post(url) nimmt eine URL zu einem öffentlichen Instagram-Beitrag entgegen und automatisiert den Zugriff darauf. Dazu wird ein Headless-Chrome-Browser im Hintergrund gestartet, um die angegebene Seite zu laden und die dort eingebetteten Inhalte auszulesen. Die Funktion wartet zunächst auf das vollständige Laden des Beitrags und versucht anschließend, ein ggf. erscheinendes Login-Popup automatisiert zu schließen, um eine ungehinderte Datenerfassung zu ermöglichen.

Es werden folgende Informationen extrahiert: der Benutzername des veröffentlichenden Accounts, die Bildunterschrift (Caption) sowie – sofern vorhanden – die Anzahl der Likes. Diese Daten werden zusammen mit der aufgerufenen URL und einem Zeitstempel in strukturierter Form gespeichert. Die Speicherung erfolgt in einer zentralen CSV-Datei, wobei bestehende Daten berücksichtigt und potenzielle Duplikate entfernt werden. Insgesamt erlaubt dieser Ansatz eine punktuelle Analyse individueller Inhalte, ist jedoch wie andere Selenium-basierte Verfahren anfällig für strukturelle Änderungen in der Instagram-Oberfläche, insbesondere bei dynamischen Komponenten und nicht stabilen XPath-Referenzen.

In [15]:
def scrape_instagram_post(url):
    try:
        logging.info(f"Starte Instagram-Scraping für URL: {url}")
        
        # Zielpfad vorbereiten
        csv_path = Path("../app/data/raw/instagram_data.csv")
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        # Browser initialisieren
        options = Options()
        options.add_argument("--headless=new")  # im Hintergrund ausführen
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920x1080")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        wait = WebDriverWait(driver, 15)

        driver.get(url)

        # Beitrag laden
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "article")))

        # Login-Popup schließen (falls vorhanden)
        try:
            close_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@role='dialog']//button")))
            close_btn.click()
        except:
            pass

        # Username extrahieren
        try:
            username_elem = wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//a[contains(@href, '/') and contains(@class, 'notranslate')]//span[1]")
            ))
            username = username_elem.text.strip()
        except:
            username = ""

        # Caption extrahieren
        try:
            caption_elem = wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//div[@data-testid='post-comment-root']//span")
            ))
            caption = caption_elem.text.strip()
        except:
            caption = ""

        # Likes (optional)
        try:
            likes_elem = wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//section//span[contains(text(), 'Gefällt')]")
            ))
            likes = likes_elem.text.strip()
        except:
            likes = ""

        # Timestamp + Struktur
        data = {
            "timestamp": datetime.now().isoformat(),
            "url": url,
            "username": username,
            "caption": caption,
            "likes": likes
        }

        df_new = pd.DataFrame([data])

        if csv_path.exists():
            df_existing = pd.read_csv(csv_path)
            df_new = pd.concat([df_existing, df_new], ignore_index=True)

        df_new.drop_duplicates(subset=["url", "caption", "username"], inplace=True)
        df_new.to_csv(csv_path, index=False)

        logging.info(f"Erfolgreich gespeichert unter {csv_path}")
        driver.quit()

    except Exception as e:
        logging.error(f"Fehler beim Scrapen von {url}: {e}")



In [25]:
# Beispielaufruf
scrape_instagram_post("https://www.instagram.com/p/DICWDtYNq7E/")

---

# 4. TWITTER / X


### 1. Versuch mit der offiziellen Twitter API (v2)

Zunächst wurde ein Zugriff über die offizielle Twitter Developer API (v2) realisiert, wobei ein Bearer Token über ein entsprechendes Developer-Konto eingebunden wurde. Die Verbindung wurde in der Regel erfolgreich hergestellt, jedoch resultierten sämtliche Anfragen bereits nach wenigen Requests in einem **HTTP 429 – Too Many Requests** Fehler. Selbst bei einer Limitierung auf nur drei Tweets wurde die Anfrage durch die API geblockt. Dies deutet auf eine äußerst restriktive Ratenbegrenzung hin, selbst im Rahmen von „Essential Access“-Leveln.

Außerdem sind in der kostenlos erstellbaren Developer App nur 100 Tweets abgreifbar, sodass diese Lösung, selbst wenn sie funktionieren würde, nicht sillführend währe.

### 2. Nutzung alternativer Scraping-Bibliotheken (z. B. `snscrape`)

In einem zweiten Schritt wurde die populäre Bibliothek [`snscrape`](https://github.com/JustAnotherArchivist/snscrape) eingesetzt, welche ohne API-Zugang direkt über die öffentliche Webschnittstelle von Twitter (u. a. GraphQL-Endpunkte) operiert. Dieses Vorgehen war früher eine gängige Methode zur Umgehung von Authentifizierungsbeschränkungen.

Aktuell führt jedoch auch dieser Ansatz regelmäßig zu Fehlern. Konkret wurde mehrfach der folgende Exception ausgelöst:

```
ScraperException: 4 requests to .../SearchTimeline failed, giving up.
```

Diese Meldung ist ein Hinweis darauf, dass Twitter die internen API-Endpunkte (GraphQL) entweder modifiziert oder stark abgesichert hat. `snscrape` kann infolgedessen die erwarteten Objekte nicht mehr extrahieren. Entsprechende Issues sind auch in der GitHub-Community der Bibliothek dokumentiert, eine stabile Lösung existiert derzeit nicht.



### 3. Headless-Scraping mit Playwright

Ein weiterer Ansatz zur Umgehung offizieller API-Beschränkungen bestand im Einsatz der Automatisierungsbibliothek [`Playwright`](https://playwright.dev/python/), die über einen echten Chromium-Browser Webinhalte visuell bzw. DOM-basiert extrahieren kann. Diese Methode ist grundsätzlich vielversprechend, da sie wie ein menschlicher Nutzer agiert und damit keine API-Limits verletzt.

Im konkreten Fall wurde Playwright zunächst als **synchrones Skript** innerhalb eines Jupyter Notebooks getestet. Dabei trat jedoch ein technisches Problem auf: Da Jupyter selbst auf einem laufenden `asyncio`-Event-Loop basiert, ist die Verwendung von `sync_playwright()` nicht erlaubt. Dies führte unmittelbar zu folgender Fehlermeldung:

```
Error: It looks like you are using Playwright Sync API inside the asyncio loop.
```

Als Reaktion darauf wurde auf die empfohlene **asynchrone API-Variante** (`async_playwright()`) umgestellt. Auch dieser Ansatz scheiterte jedoch – insbesondere unter Windows – mit folgendem Fehler:

```
NotImplementedError: asyncio subprocess transport is not implemented
```

Diese Meldung verweist auf eine bekannte Einschränkung der Playwright-Implementierung im Zusammenspiel mit dem Event-Loop von Windows und Jupyter. Der Start des internen Chromium-Drivers schlägt aufgrund fehlender Unterstützung für `asyncio.create_subprocess_exec()` fehl. Damit ist Playwright im Rahmen einer Notebook-Umgebung faktisch **nicht lauffähig**.

---
# 5. TikTok

In [28]:
# TikTokApi ist eine Bibliothek zum Abrufen von TikTok-Daten (inoffizielle API)
from TikTokApi import TikTokApi

In [29]:
# Token für die Session – muss aus Cookies der TikTok website gezogen werden
ms_token = os.getenv("MS_TOKEN")  

# Pfad zur Ausgabedatei (CSV), Standardwert: "trending_videos.csv"
csv_path = os.getenv("OUTPUT_PATH", "trending_videos.csv")

Der nachfolgende Code nutzt nun die inoffizielle TikTokApi, um die aktuell beliebtesten TikTok-Videos (Trending-Videos) automatisiert abzurufen. Mithilfe des gültigen ms_token (Session-Token) wird eine Browser-Sitzung erstellt, die den Zugriff auf öffentliche TikTok-Inhalte ermöglicht. Die Funktion extrahiert Informationen zu 30 populären Videos – darunter Beschreibung, Autor, Likes, Views und URL – und speichert die Ergebnisse in einer CSV-Datei. Die gesamte Logik läuft asynchron ab, um die Effizienz bei der Netzwerkkommunikation zu verbessern.

In [30]:
async def trending_videos():
    # TikTok-API-Session asynchron starten
    async with TikTokApi() as api:
        await api.create_sessions(
            ms_tokens=[ms_token],  # Session-Token für Authentifizierung
            num_sessions=1,        # Anzahl gleichzeitiger Sessions
            sleep_after=3,         # Wartezeit zwischen Session-Anfragen
            browser=os.getenv("TIKTOK_BROWSER", "chromium")  # Standardbrowser
        )

        data = []

        # Schleife über die 30 beliebtesten Trending-Videos
        async for video in api.trending.videos(count=30):
            info = video.as_dict  # Rohdaten des Videos als Dictionary

            # Relevante Felder extrahieren und speichern
            data.append({
                "id": info.get("id"),
                "description": info.get("desc"),
                "author_username": info.get("author", {}).get("uniqueId"),
                "author_id": info.get("author", {}).get("id"),
                "likes": info.get("stats", {}).get("diggCount"),
                "shares": info.get("stats", {}).get("shareCount"),
                "comments": info.get("stats", {}).get("commentCount"),
                "plays": info.get("stats", {}).get("playCount"),
                "video_url": info.get("video", {}).get("downloadAddr"),
                "created_time": info.get("createTime"),
            })

        # Sicherstellen, dass der Zielordner existiert
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)

        # Ergebnisse in CSV-Datei schreiben (append-Modus)
        file_exists = os.path.isfile(csv_path)
        with open(csv_path, "a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=data[0].keys())

            if not file_exists:
                writer.writeheader()  # Schreibe Header nur, wenn Datei neu ist
            writer.writerows(data)   # Anhängen der neuen Zeilen

        print(f"\n✅ Erfolgreich {len(data)} Videos in '{csv_path}' gespeichert.")


---
# 6. YouTube

Um YouTube zu scrapen wird die offizielle YouTube Data API v3 genutzt, um die derzeit beliebtesten Videos (Trending) aus einer bestimmten Region abzurufen. Es extrahiert Titel, Beschreibung, Kanalname, Veröffentlichungsdatum sowie Statistiken wie Aufrufe, Likes und Kommentare. Die Daten werden als Pandas DataFrame verarbeitet und in einer CSV-Datei gespeichert. Bestehende Daten werden berücksichtigt, Duplikate entfernt und das Ergebnis ausgegeben.

In [31]:
#offizieller Google API Client für Python
from googleapiclient.discovery import build

In [32]:
# API Set-up
API_KEY = os.getenv("YT_KEY")  # API-Key aus Umgebungsvariablen
youtube = build("youtube", "v3", developerKey=API_KEY)  # YouTube-API-Client erzeugen

In [33]:
def scrape_youtube_trending(region="DE", max_results=50):
    try:
        print("🔍 Starte YouTube-Scraping...")

        # API-Anfrage vorbereiten (meistgesehene Videos in gegebener Region)
        request = youtube.videos().list(
            part="snippet,statistics",
            chart="mostPopular",
            regionCode=region,
            maxResults=max_results
        )
        response = request.execute()  # Anfrage ausführen

        videos = []
        scrape_time = datetime.now()  # Zeitstempel des Abrufs

        # Schleife über alle zurückgegebenen Videos
        for item in response.get("items", []):
            snippet = item["snippet"]
            stats = item.get("statistics", {})

            videos.append({
                "video_id": item["id"],
                "title": snippet.get("title"),
                "description": snippet.get("description"),
                "channel_title": snippet.get("channelTitle"),
                "published_at": snippet.get("publishedAt"),
                "view_count": stats.get("viewCount"),
                "like_count": stats.get("likeCount"),
                "comment_count": stats.get("commentCount"),
                "url": f"https://www.youtube.com/watch?v={item['id']}",
                "scraped_at": scrape_time
            })

        # In DataFrame umwandeln
        df = pd.DataFrame(videos)

        # CSV-Pfad vorbereiten
        csv_path = Path("../app/data/raw/youtube_data.csv")
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        # Falls Datei bereits existiert → zusammenführen
        if csv_path.exists():
            df_existing = pd.read_csv(csv_path)
            df = pd.concat([df_existing, df], ignore_index=True)

        # Duplikate entfernen und speichern
        df.drop_duplicates(subset=["video_id"], inplace=True)
        df.to_csv(csv_path, index=False)

        # Vorschau der Daten anzeigen
        display(df.head())

        print(f"✅ {len(df)} Videos gespeichert unter: {csv_path}")

    except Exception as e:
        print(f"❌ Fehler beim YouTube-Scraping: {e}")


In [34]:
if __name__ == "__main__":
    scrape_youtube_trending()


🔍 Starte YouTube-Scraping...


,video_id,title,description,channel_title,published_at,view_count,like_count,comment_count,url,scraped_at
0,mLpa3zzrVyI,Inter Mailand 4:3 FC Barcelona | Highlights - ...,Viel Spaß mit den Highlights des Spiels Inter ...,Prime Video Sport Deutschland,2025-05-06T21:48:35Z,1375723,25099,2585,https://www.youtube.com/watch?v=mLpa3zzrVyI,2025-05-07 14:12:27.972453
1,nRw_GB_78xQ,Drei Festnahmen & eine Waffe | Nachtschicht mi...,WERBUNG Mit dem Code HEYAARON kann Finanzguru ...,Hey Aaron!!!,2025-05-06T16:01:28Z,319867,17503,338,https://www.youtube.com/watch?v=nRw_GB_78xQ,2025-05-07 14:12:27.972453
2,SJP2M5IumRc,Merz scheitert im 1. Wahlgang - und ist jetzt ...,Friedrich Merz ist neuer Kanzler der Bundesrep...,MrWissen2go,2025-05-06T15:21:05Z,430465,20861,3085,https://www.youtube.com/watch?v=SJP2M5IumRc,2025-05-07 14:12:27.972453
3,m5GRR4D23k0,Was ich euch sagen möchte...,Mein zweiter Kanal: https://www.youtube.com/@A...,SkylineTV,2025-05-06T15:15:05Z,129142,9029,1812,https://www.youtube.com/watch?v=m5GRR4D23k0,2025-05-07 14:12:27.972453
4,HwMeliz7paA,IN 7 TAGEN 491 KM 🏃🏻‍♂️🤯| I run the full lengt...,Instagram: https://www.instagram.com/ardasaatc...,Arda Saatçi,2025-05-06T16:59:06Z,286018,27371,992,https://www.youtube.com/watch?v=HwMeliz7paA,2025-05-07 14:12:27.972453


✅ 50 Videos gespeichert unter: ..\app\data\raw\youtube_data.csv


---
# 7. Fazit

Das Scraping für die Plattformen **Reddit**, **TikTok** und **YouTube** wurde erfolgreich implementiert. Für jede dieser Plattformen wurden funktionierende Ansätze entwickelt, die auf öffentlich zugängliche APIs oder geeignete inoffizielle Schnittstellen zurückgreifen. Die lauffähigen und getesteten Skripte befinden sich im Verzeichnis **`src/1_scheduler/jobs`** und können dort für die regelmäßige Datenerhebung genutzt werden.
Für **X (ehemals Twitter)** und **Instagram** konnte hingegen kein stabiles und zuverlässiges Scraping-Verfahren umgesetzt werden, da beide Plattformen den Zugriff durch technische und rechtliche Maßnahmen erheblich einschränken.
